In [1]:
from __future__ import division, print_function
from gensim import models
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, Reshape, Flatten, concatenate, Input, Conv1D, GlobalMaxPooling1D, Embedding
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import os
import collections
import re
import string

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
data = pd.read_csv('/content/drive/My Drive/Reviews.txt', delimiter='\t')

In [0]:
data.columns = ['Text','Label']

In [5]:
data.head()

,Text,Label
0,Not sure who was more lost - the flat characte...,0
1,Attempting artiness with black & white and cle...,0
2,Very little music or anything to speak of.,0
3,The best scene in the movie was when Gerardo i...,1
4,"The rest of the movie lacks art, charm, meanin...",0


In [6]:
data.Label.unique()

array([0, 1])

In [7]:
data.shape

(747, 2)

In [0]:
pos = []
neg = []
for l in data.Label:
    if l == 0 :
        pos.append(0)
        neg.append(1)
    elif l == 1:
        pos.append(1)
        neg.append(0)          

In [0]:
data['Pos']= pos
data['Neg']= neg

In [10]:
data.head()

,Text,Label,Pos,Neg
0,Not sure who was more lost - the flat characte...,0,0,1
1,Attempting artiness with black & white and cle...,0,0,1
2,Very little music or anything to speak of.,0,0,1
3,The best scene in the movie was when Gerardo i...,1,1,0
4,"The rest of the movie lacks art, charm, meanin...",0,0,1


In [11]:
data.tail()

,Text,Label,Pos,Neg
742,I just got bored watching Jessice Lange take h...,0,0,1
743,"Unfortunately, any virtue in this film's produ...",0,0,1
744,"In a word, it is embarrassing.",0,0,1
745,Exceptionally bad!,0,0,1
746,All in all its an insult to one's intelligence...,0,0,1


In [0]:
def remove_punct(text):
    text_nopunct = ''
    text_nopunct = re.sub('['+string.punctuation+']', '', text)
    return text_nopunct

data['Text_Clean'] = data['Text'].apply(lambda x: remove_punct(x))

In [13]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
from nltk import word_tokenize, WordNetLemmatizer
tokens = [word_tokenize(sen) for sen in data.Text_Clean]

In [0]:
def lower_token(tokens): 
    return [w.lower() for w in tokens]    
    
lower_tokens = [lower_token(token) for token in tokens]

In [16]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
from nltk.corpus import stopwords
stoplist = stopwords.words('english')

In [0]:
def remove_stop_words(tokens): 
    return [word for word in tokens if word not in stoplist]

In [0]:
filtered_words = [remove_stop_words(sen) for sen in lower_tokens]

In [0]:
result = [' '.join(sen) for sen in filtered_words]

In [0]:
data['Text_Final'] = result

In [0]:
data['tokens'] = filtered_words

In [0]:
data = data[['Text_Final', 'tokens', 'Label', 'Pos', 'Neg']]

In [24]:
data[:10]

,Text_Final,tokens,Label,Pos,Neg
0,sure lost flat characters audience nearly half...,"[sure, lost, flat, characters, audience, nearl...",0,0,1
1,attempting artiness black white clever camera ...,"[attempting, artiness, black, white, clever, c...",0,0,1
2,little music anything speak,"[little, music, anything, speak]",0,0,1
3,best scene movie gerardo trying find song keep...,"[best, scene, movie, gerardo, trying, find, so...",1,1,0
4,rest movie lacks art charm meaning emptiness w...,"[rest, movie, lacks, art, charm, meaning, empt...",0,0,1
5,wasted two hours,"[wasted, two, hours]",0,0,1
6,saw movie today thought good effort good messa...,"[saw, movie, today, thought, good, effort, goo...",1,1,0
7,bit predictable,"[bit, predictable]",0,0,1
8,loved casting jimmy buffet science teacher,"[loved, casting, jimmy, buffet, science, teacher]",1,1,0
9,baby owls adorable,"[baby, owls, adorable]",1,1,0


In [0]:
data_train, data_test = train_test_split(data, test_size=0.10, random_state=42)

In [26]:
all_training_words = [word for tokens in data_train["tokens"] for word in tokens]
training_sentence_lengths = [len(tokens) for tokens in data_train["tokens"]]
TRAINING_VOCAB = sorted(list(set(all_training_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_training_words), len(TRAINING_VOCAB)))
print("Max sentence length is %s" % max(training_sentence_lengths))

6968 words total, with a vocabulary size of 2824
Max sentence length is 789


In [27]:
all_test_words = [word for tokens in data_test["tokens"] for word in tokens]
test_sentence_lengths = [len(tokens) for tokens in data_test["tokens"]]
TEST_VOCAB = sorted(list(set(all_test_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_test_words), len(TEST_VOCAB)))
print("Max sentence length is %s" % max(test_sentence_lengths))

823 words total, with a vocabulary size of 584
Max sentence length is 160


In [28]:
word2vec_path = '/content/drive/My Drive/envec.txt'
word2vec = models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
def get_average_word2vec(tokens_list, vector, generate_missing=False, k=200):
    if len(tokens_list)<1:
        return np.zeros(k)
    if generate_missing:
        vectorized = [vector[word] if word in vector else np.random.rand(k) for word in tokens_list]
    else:
        vectorized = [vector[word] if word in vector else np.zeros(k) for word in tokens_list]
    length = len(vectorized)
    summed = np.sum(vectorized, axis=0)
    averaged = np.divide(summed, length)
    return averaged

def get_word2vec_embeddings(vectors, clean_comments, generate_missing=False):
    embeddings = clean_comments['tokens'].apply(lambda x: get_average_word2vec(x, vectors, 
                                                                                generate_missing=generate_missing))
    return list(embeddings)

In [0]:
training_embeddings = get_word2vec_embeddings(word2vec, data_train, generate_missing=True)

In [0]:
MAX_SEQUENCE_LENGTH = 50
EMBEDDING_DIM = 200

In [32]:
tokenizer = Tokenizer(num_words=len(TRAINING_VOCAB), lower=True, char_level=False)
tokenizer.fit_on_texts(data_train["Text_Final"].tolist())
training_sequences = tokenizer.texts_to_sequences(data_train["Text_Final"].tolist())

train_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(train_word_index))

Found 2824 unique tokens.


In [0]:
train_cnn_data = pad_sequences(training_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [34]:
train_embedding_weights = np.zeros((len(train_word_index)+1, EMBEDDING_DIM))
for word,index in train_word_index.items():
    train_embedding_weights[index,:] = word2vec[word] if word in word2vec else np.random.rand(EMBEDDING_DIM)
print(train_embedding_weights.shape)

(2825, 200)


In [0]:
test_sequences = tokenizer.texts_to_sequences(data_test["Text_Final"].tolist())
test_cnn_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [0]:
def ConvNet(embeddings, max_sequence_length, num_words, embedding_dim, labels_index):
    
    embedding_layer = Embedding(num_words,
                            embedding_dim,
                            weights=[embeddings],
                            input_length=max_sequence_length,
                            trainable=False)
    
    sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)

    convs = []
    filter_sizes = [2,3,4,5,6]

    for filter_size in filter_sizes:
        l_conv = Conv1D(filters=200, kernel_size=filter_size, activation='relu')(embedded_sequences)
        l_pool = GlobalMaxPooling1D()(l_conv)
        convs.append(l_pool)


    l_merge = concatenate(convs, axis=1)

    x = Dropout(0.1)(l_merge)  
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    preds = Dense(labels_index, activation='sigmoid')(x)

    model = Model(sequence_input, preds)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])
    model.summary()
    return model

In [0]:
label_names = ['Pos', 'Neg']

In [0]:
y_train = data_train[label_names].values

In [0]:
x_train = train_cnn_data
y_tr = y_train

In [40]:
model = ConvNet(train_embedding_weights, MAX_SEQUENCE_LENGTH, len(train_word_index)+1, EMBEDDING_DIM, 
                len(list(label_names)))











Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 50)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 50, 200)      565000      input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 49, 200)      80200       embedding_1[0][0]                
_____________________________________________________________________

In [0]:
num_epochs = 100
batch_size = 34

In [42]:
hist = model.fit(x_train, y_tr, epochs=num_epochs, validation_split=0.1, shuffle=True, batch_size=batch_size)



Train on 604 samples, validate on 68 samples
Epoch 1/100
604/604 [==============================] - 7s 12ms/step - loss: 0.9672 - acc: 0.5124 - val_loss: 0.6946 - val_acc: 0.5074
Epoch 2/100
604/604 [==============================] - 0s 224us/step - loss: 0.8242 - acc: 0.5629 - val_loss: 0.6788 - val_acc: 0.5588
Epoch 3/100
604/604 [==============================] - 0s 210us/step - loss: 0.8231 - acc: 0.5844 - val_loss: 0.7281 - val_acc: 0.4412
Epoch 4/100
604/604 [==============================] - 0s 221us/step - loss: 0.7821 - acc: 0.5935 - val_loss: 0.6808 - val_acc: 0.5662
Epoch 5/100
604/604 [==============================] - 0s 206us/step - loss: 0.7987 - acc: 0.6391 - val_loss: 0.6586 - val_acc: 0.6985
Epoch 6/100
604/604 [==============================] - 0s 225us/step - loss: 0.7317 - acc: 0.6879 - val_loss: 0.6936 - val_acc: 0.4779
Epoch 7/100
604/604 [==============================] - 0s 234us/step - loss: 0.6925 - acc: 0.7219 - val_loss: 0.6459 - val_acc: 0.6618
Epoch 8/1

In [43]:
predictions = model.predict(test_cnn_data, batch_size=1024, verbose=1)

75/75 [==============================] - 0s 3ms/step


In [0]:
labels = [1, 0]

In [0]:
prediction_labels=[]
for p in predictions:
    prediction_labels.append(labels[np.argmax(p)])

In [46]:
sum(data_test.Label==prediction_labels)/len(prediction_labels)

0.6666666666666666

In [47]:
data_test.Label.value_counts()

0    39
1    36
Name: Label, dtype: int64